In [2]:
import pandas as pd

In [122]:
dfelec = pd.read_csv('ElectricityPriceNov.csv')
dfcons= pd.read_csv('Aggregated Building Data.csv')
dfgen= pd.read_csv('Aggregated Solar Data.csv')

# For Battery Level Graph

In [280]:
dfelec = pd.read_csv('ElectricityPriceNov.csv')
dfcons= pd.read_csv('Aggregated Building Data.csv')
dfgen= pd.read_csv('Aggregated Solar Data.csv')


battery_capacity = 10
release_battery = battery_capacity



elecsaved = 0
costsaved = 0
battery_levels = []

total_charge_used=0
total_money_saved=0

for i in range(0,721,24):
    tmp = [0 for i in range(24)]
    
    if i == 0:
        pass
    
    elif i == 720:
        
        charge_used_perday=0
        money_saved=0
        lst=[i-p for p in range(1,25)]
        
        while (len(lst)!=0) & (release_battery-charge_used_perday>0):
            #print(lst)
            maximum=max(max(dfelec['AvgRRP'][(i-24):i]),0) #maximum price in given window
            
            dfelecwindow = pd.DataFrame(dfelec['AvgRRP'][(i-24):(i)])
            
            #print('inner maximum price',maximum)
            
            #Handle negative prices
            if maximum == 0:
                break
            
            index = dfelecwindow.loc[(dfelecwindow['AvgRRP']==maximum)].index[0] 
            maxeleccons = dfcons.loc[[index], 'prediction'][index]
            #print('inner maxmimum price index ',index)
            
            
            charge_used_perhr = min(maxeleccons, release_battery-charge_used_perday)
            #print('inner max elec cons ',maxeleccons)
            #print('release_battery - charge_used_perday ', (release_battery-charge_used_perday))
            #print('charge_used_perhr is ', charge_used_perhr)
            
            charge_used_perday += charge_used_perhr
            money_saved += maximum*charge_used_perhr
            
            
            #print(index)
            lst.remove(index)
            dfelec.loc[[index], 'AvgRRP'] = 0
            tmp[index%24] = charge_used_perhr
        
        release_battery -= charge_used_perday
        total_charge_used += charge_used_perday
        total_money_saved += money_saved
        battery_levels.append(tmp)
    

    else:
        
        charge_used_perday=0
        money_saved=0
        
        maximum=max(dfelec['AvgRRP'][(i-24):(i)]) #maximum electricity price
        #print('outer max elec price ', maximum)
        
        dfelecwindow = pd.DataFrame(dfelec['AvgRRP'][(i-24):(i)])
        
        index = dfelecwindow.loc[(dfelecwindow['AvgRRP']==maximum)].index[0] #maximum electricity price index
        print('outer index is ', index)
        
        maxeleccons = dfcons.loc[[index], 'prediction'][index]
        #print('outer max elec cons is ', maxeleccons)
        
        #Get next day's index and max electricity consumption
        nxt_maximum=max(dfelec['AvgRRP'][(i):(i+24)])
        
        dfelecnxtwindow = pd.DataFrame(dfelec['AvgRRP'][(i):(i+24)])
        
        nxt_index= dfelecnxtwindow.loc[(dfelecnxtwindow['AvgRRP']==nxt_maximum)].index[0]
        #print('outer Next elec price max ', nxt_maximum)
        print('outer Next elec price max index ', nxt_index)
        
        solargen=sum(dfgen['prediction'][index:nxt_index])
        #print('outer solar generation', solargen)
        
        lst=[i-p for p in range(1,25)]
        
        while  (charge_used_perday < solargen) & (len(lst)!=0) & (battery_capacity-charge_used_perday>0):
            #print('inner charged used per day', charge_used_perday)
            
            #print(lst)
            
            maximum=max(max(dfelec['AvgRRP'][(i-24):i]),0)
            #print('inner maximum price',maximum)
            #Handle negative prices
            if maximum == 0:
                break
            
            index = dfelecwindow.loc[(dfelecwindow['AvgRRP']==maximum)].index[0] 
            maxeleccons = dfcons.loc[[index], 'prediction'][index]
            #print('inner maxmimum price index ',index)
            
            
            charge_used_perhr = min((solargen-charge_used_perday), maxeleccons, battery_capacity-charge_used_perday)
            #print('inner max elec cons ',maxeleccons)
            #print('inner solar - charge day ', (solargen-charge_used_perday))
            #print('battery_capacity - charge_used_perday ', (battery_capacity-charge_used_perday))
            #print('inner charge used per hour ', charge_used_perhr)
            
            charge_used_perday += charge_used_perhr
            money_saved += maximum*charge_used_perhr
            
            lst.remove(index)
            dfelec.loc[[index], 'AvgRRP'] = 0
            tmp[index%24] = charge_used_perhr
        
        total_charge_used += charge_used_perday
        total_money_saved += money_saved
        battery_levels.append(tmp)
        
        #print('tote charge save',total_charge_used)
        #print('tote mon save', total_money_saved)

print(total_charge_used)
print(total_money_saved)
print(battery_levels)

outer index is  18
outer Next elec price max index  42
outer index is  42
outer Next elec price max index  65
outer index is  65
outer Next elec price max index  89
outer index is  89
outer Next elec price max index  114
outer index is  114
outer Next elec price max index  126
outer index is  126
outer Next elec price max index  144
outer index is  144
outer Next elec price max index  186
outer index is  186
outer Next elec price max index  210
outer index is  210
outer Next elec price max index  234
outer index is  234
outer Next elec price max index  246
outer index is  246
outer Next elec price max index  281
outer index is  281
outer Next elec price max index  306
outer index is  306
outer Next elec price max index  331
outer index is  331
outer Next elec price max index  354
outer index is  354
outer Next elec price max index  379
outer index is  379
outer Next elec price max index  402
outer index is  402
outer Next elec price max index  425
outer index is  425
outer Next elec pr

# For Electrcity & Cost Saved Hourly

In [285]:
dfelec = pd.read_csv('ElectricityPriceNov.csv')
dfcons= pd.read_csv('Aggregated Building Data.csv')
dfgen= pd.read_csv('Aggregated Solar Data.csv')


battery_capacity = 10
release_battery = battery_capacity



elecsaved = 0
costsaved = 0
battery_levels = []

total_charge_used=0
total_money_saved=0

elec_lst=[]
cost_lst=[]

for i in range(0,721,24):
    tmp = [0 for i in range(24)]
    elec_lst_tmp = [0 for i in range(24)]
    cost_lst_tmp = [0 for i in range(24)]
    
    if i == 0:
        pass
    
    elif i == 720:
        
        charge_used_perday=0
        money_saved=0
        lst=[i-p for p in range(1,25)]
        
        while (len(lst)!=0) & (release_battery-charge_used_perday>0):
            #print(lst)
            maximum=max(max(dfelec['AvgRRP'][(i-24):i]),0) #maximum price in given window
            
            dfelecwindow = pd.DataFrame(dfelec['AvgRRP'][(i-24):(i)])
            
            #print('inner maximum price',maximum)
            
            #Handle negative prices
            if maximum == 0:
                break
            
            index = dfelecwindow.loc[(dfelecwindow['AvgRRP']==maximum)].index[0] 
            maxeleccons = dfcons.loc[[index], 'prediction'][index]
            #print('inner maxmimum price index ',index)
            
            
            charge_used_perhr = min(maxeleccons, release_battery-charge_used_perday)
            #print('inner max elec cons ',maxeleccons)
            #print('release_battery - charge_used_perday ', (release_battery-charge_used_perday))
            #print('charge_used_perhr is ', charge_used_perhr)
            
            charge_used_perday += charge_used_perhr
            money_saved += maximum*charge_used_perhr
            
            
            #print(index)
            lst.remove(index)
            dfelec.loc[[index], 'AvgRRP'] = 0
            tmp[index%24] = charge_used_perhr
            elec_lst_tmp[index%24] = charge_used_perhr
            cost_lst_tmp[index%24] = maximum*charge_used_perhr
        
        release_battery -= charge_used_perday
        total_charge_used += charge_used_perday
        total_money_saved += money_saved
        
        battery_levels.append(tmp)
        cost_lst.append(cost_lst_tmp)
        elec_lst.append(elec_lst_tmp)
    

    else:
        
        charge_used_perday=0
        money_saved=0
        
        maximum=max(dfelec['AvgRRP'][(i-24):(i)]) #maximum electricity price
        #print('outer max elec price ', maximum)
        
        dfelecwindow = pd.DataFrame(dfelec['AvgRRP'][(i-24):(i)])
        
        index = dfelecwindow.loc[(dfelecwindow['AvgRRP']==maximum)].index[0] #maximum electricity price index
        #print('outer index is ', index)
        
        maxeleccons = dfcons.loc[[index], 'prediction'][index]
        #print('outer max elec cons is ', maxeleccons)
        
        #Get next day's index and max electricity consumption
        nxt_maximum=max(dfelec['AvgRRP'][(i):(i+24)])
        
        dfelecnxtwindow = pd.DataFrame(dfelec['AvgRRP'][(i):(i+24)])
        
        nxt_index= dfelecnxtwindow.loc[(dfelecnxtwindow['AvgRRP']==nxt_maximum)].index[0]
        #print('outer Next elec price max ', nxt_maximum)
        #print('outer Next elec price max index ', nxt_index)
        
        solargen=sum(dfgen['prediction'][index:nxt_index])
        #print('outer solar generation', solargen)
        
        lst=[i-p for p in range(1,25)]
        
        while  (charge_used_perday < solargen) & (len(lst)!=0) & (battery_capacity-charge_used_perday>0):
            #print('inner charged used per day', charge_used_perday)
            
            #print(lst)
            
            maximum=max(max(dfelec['AvgRRP'][(i-24):i]),0)
            #print('inner maximum price',maximum)
            #Handle negative prices
            if maximum == 0:
                break
            
            index = dfelecwindow.loc[(dfelecwindow['AvgRRP']==maximum)].index[0] 
            maxeleccons = dfcons.loc[[index], 'prediction'][index]
            #print('inner maxmimum price index ',index)
            
            
            charge_used_perhr = min((solargen-charge_used_perday), maxeleccons, battery_capacity-charge_used_perday)
            #print('inner max elec cons ',maxeleccons)
            #print('inner solar - charge day ', (solargen-charge_used_perday))
            #print('battery_capacity - charge_used_perday ', (battery_capacity-charge_used_perday))
            #print('inner charge used per hour ', charge_used_perhr)
            
            charge_used_perday += charge_used_perhr
            money_saved += maximum*charge_used_perhr
            
            lst.remove(index)
            dfelec.loc[[index], 'AvgRRP'] = 0
            tmp[index%24] = charge_used_perhr
            elec_lst_tmp[index%24] = charge_used_perhr
            cost_lst_tmp[index%24] = maximum*charge_used_perhr
        
        total_charge_used += charge_used_perday
        total_money_saved += money_saved
        battery_levels.append(tmp)
        
        elec_lst.append(elec_lst_tmp)
        cost_lst.append(cost_lst_tmp)
        
        
        #print('tote charge save',total_charge_used)
        #print('tote mon save', total_money_saved)

#print(total_charge_used)
#print(total_money_saved)

print(elec_lst)
print(cost_lst)
#print(battery_levels)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0,

# Original Optimisation

In [286]:
dfelec = pd.read_csv('ElectricityPriceNov.csv')
dfcons= pd.read_csv('Aggregated Building Data.csv')
dfgen= pd.read_csv('Aggregated Solar Data.csv')


battery_capacity = 10
release_battery = battery_capacity



elecsaved = 0
costsaved = 0
battery_levels = []

total_charge_used=0
total_money_saved=0

for i in range(0,721,24):
    if i == 0:
        pass
    
    elif i == 720:
        
        charge_used_perday=0
        money_saved=0
        lst=[i-p for p in range(1,25)]
        
        while (len(lst)!=0) & (release_battery-charge_used_perday>0):
            
            maximum=max(max(dfelec['AvgRRP'][(i-24):i]),0) #maximum price in given window
            
            dfelecwindow = pd.DataFrame(dfelec['AvgRRP'][(i-24):(i)])
            
            #print('inner maximum price',maximum)
            
            #Handle negative prices
            if maximum == 0:
                break
            
            index = dfelecwindow.loc[(dfelecwindow['AvgRRP']==maximum)].index[0] 
            maxeleccons = dfcons.loc[[index], 'prediction'][index]
            #print('inner maxmimum price index ',index)
            
            
            charge_used_perhr = min(maxeleccons, release_battery-charge_used_perday)
            #print('inner max elec cons ',maxeleccons)
            #print('release_battery - charge_used_perday ', (release_battery-charge_used_perday))
            #print('charge_used_perhr is ', charge_used_perhr)
            
            charge_used_perday += charge_used_perhr
            money_saved += maximum*charge_used_perhr
            
            
            #print(index)
            lst.remove(index)
            dfelec.loc[[index], 'AvgRRP'] = 0
        
        release_battery -= charge_used_perday
        total_charge_used += charge_used_perday
        total_money_saved += money_saved
    

    else:
        
        charge_used_perday=0
        money_saved=0
        
        maximum=max(dfelec['AvgRRP'][(i-24):(i)]) #maximum electricity price
        #print('outer max elec price ', maximum)
        
        dfelecwindow = pd.DataFrame(dfelec['AvgRRP'][(i-24):(i)])
        
        index = dfelecwindow.loc[(dfelecwindow['AvgRRP']==maximum)].index[0] #maximum electricity price index
        #print('outer index is ', index)
        
        maxeleccons = dfcons.loc[[index], 'prediction'][index]
        #print('outer max elec cons is ', maxeleccons)
        
        #Get next day's index and max electricity consumption
        nxt_maximum=max(dfelec['AvgRRP'][(i):(i+24)])
        
        dfelecnxtwindow = pd.DataFrame(dfelec['AvgRRP'][(i):(i+24)])
        
        nxt_index= dfelecnxtwindow.loc[(dfelecnxtwindow['AvgRRP']==nxt_maximum)].index[0]
        #print('outer Next elec price max ', nxt_maximum)
        #print('outer Next elec price max index ', nxt_index)
        
        solargen=sum(dfgen['prediction'][index:nxt_index])
        #print('outer solar generation', solargen)
        
        lst=[i-p for p in range(1,25)]
        
        while  (charge_used_perday < solargen) & (len(lst)!=0) & (battery_capacity-charge_used_perday>0):
            #print('inner charged used per day', charge_used_perday)
            
            print(lst)
            
            maximum=max(max(dfelec['AvgRRP'][(i-24):i]),0)
            #print('inner maximum price',maximum)
            #Handle negative prices
            if maximum == 0:
                break
            
            index = dfelecwindow.loc[(dfelecwindow['AvgRRP']==maximum)].index[0] 
            maxeleccons = dfcons.loc[[index], 'prediction'][index]
            #print('inner maxmimum price index ',index)
            
            
            charge_used_perhr = min((solargen-charge_used_perday), maxeleccons, battery_capacity-charge_used_perday)
            #print('inner max elec cons ',maxeleccons)
            #print('inner solar - charge day ', (solargen-charge_used_perday))
            #print('battery_capacity - charge_used_perday ', (battery_capacity-charge_used_perday))
            #print('inner charge used per hour ', charge_used_perhr)
            
            charge_used_perday += charge_used_perhr
            money_saved += maximum*charge_used_perhr
            
            lst.remove(index)
            dfelec.loc[[index], 'AvgRRP'] = 0
        
        total_charge_used += charge_used_perday
        total_money_saved += money_saved
        
        #print('tote charge save',total_charge_used)
        #print('tote mon save', total_money_saved)

print(total_charge_used)
print(total_money_saved)

[23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
[47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25, 24]
[71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48]
[95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72]
[119, 118, 117, 116, 115, 114, 113, 112, 111, 110, 109, 108, 107, 106, 105, 104, 103, 102, 101, 100, 99, 98, 97, 96]
[143, 142, 141, 140, 139, 138, 137, 136, 135, 134, 133, 132, 131, 130, 129, 128, 127, 126, 125, 124, 123, 122, 121, 120]
[167, 166, 165, 164, 163, 162, 161, 160, 159, 158, 157, 156, 155, 154, 153, 152, 151, 150, 149, 148, 147, 146, 145, 144]
[191, 190, 189, 188, 187, 186, 185, 184, 183, 182, 181, 180, 179, 178, 177, 176, 175, 174, 173, 172, 171, 170, 169, 168]
[215, 214, 213, 212, 211, 210, 209, 208, 207, 206, 205, 204, 203, 202, 201, 200, 199, 198, 197, 196, 195, 194, 193, 192]
[239, 238, 237, 236, 